In [1]:
%reload_ext autoreload
%autoreload 1
%aimport train

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
# tweet_df, vocab = train.load_data(num_docs=10000)
# embedding_index_map = tweet_df[['id', 'text']]
# embedding_index_map.to_pickle('embedding_index_map.pkl')

In [4]:
embedding_index_map = pd.read_pickle('embedding_index_map.pkl')

In [5]:
import pickle
with open('tweet_vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
    vocab = np.array(vocab)

In [6]:
# docs, contexts, targets = train.prepare_contexts(tweet_df)

In [7]:
# tf.reset_default_graph()

In [8]:
# model = train.build_model(num_words = len(vocab), num_docs = len(tweet_df))

# model = train.build_model(num_words = len(vocab), num_docs = len(tweet_df),
#                           doc_embedding_size=20, word_embedding_size=50)

In [9]:
# model.load('nce_test_model-10371')

In [10]:
# e1, = model.net.graph.get_collection('layer_variables/Embedding/')
# e2, = model.net.graph.get_collection('layer_variables/Embedding_1/')

# doc_embedding = model.get_weights(e1)
# word_embedding = model.get_weights(e2)

In [11]:
# train.train_model(model, docs, contexts, targets, num_words = len(vocab))

In [12]:
trainer, X1, X2, Y = train.build_nce_model(num_words = len(vocab), num_docs = len(embedding_index_map))

In [13]:
trainer.restore('nce_test_model-46674')

In [14]:
e1, = trainer.graph.get_collection('layer_variables/Embedding/')
e2, = trainer.graph.get_collection('layer_variables/Embedding_1/')

doc_embedding = e1.eval(session=trainer.session)
word_embedding = e2.eval(session=trainer.session)

In [16]:
doc_embedding_series = pd.Series(list(doc_embedding))
embedding_index_map['embedding'] = doc_embedding_series

In [17]:
doc_embedding = np.stack(embedding_index_map.embedding.values)

In [18]:
doc_embedding_norm = doc_embedding / np.linalg.norm(doc_embedding, axis=1, keepdims=True)

In [19]:
doc = doc_embedding_norm[2011]
doc = doc_embedding_norm[2012]
doc = doc_embedding_norm[2013]
doc = doc_embedding_norm[3008]
# doc = doc_embedding_norm[3009]
# doc = doc_embedding_norm[3012]
# doc = doc_embedding_norm[4000]
# doc = doc_embedding_norm[4005]
# doc = doc_embedding_norm[5000]



dist = np.sum(doc * doc_embedding_norm, axis=1)
dist_sort_idx = np.argsort(dist)[::-1]
for i in dist_sort_idx[:25]:
    t = embedding_index_map.iloc[i].text
    d = dist[i]
    print('---', np.round(d, 2))
    print(t)

--- 1.0
RT @JulieBishopMP: My statement tonight in response to decision in Philippines arbitration case on South China Sea https://t.co/H7e1tjaace
--- 0.96
My statement tonight in response to decision in Philippines arbitration case on South China Sea https://t.co/H7e1tjaace
--- 0.73
RT @geoff_p_wade: India Assisting Philippines in Getting Warship to Patrol South China Sea
https://t.co/e1d5ght64U https://t.co/bBb5cq1c2D
--- 0.69
My statement regarding the unfolding situation in Turkey https://t.co/TVKRsZETwG
--- 0.67
Philippines wins South China Sea case against China.Fasten seat belts @spellar @9DashLine @wildgees8 @narendramodi  https://t.co/NfPEAu9gVi
--- 0.66
RT @CAREAustralia: MEDIA RELEASE: CARE statement on outbreak of violence in South Sudan https://t.co/Ng3SswZd4C #SouthSudan
--- 0.64
RT @JosephScales: Territorians Demand Better
@ASUnion statement on @4corners' "Australia's shame" #4Corners #ntpol #auspol https://t.co/gXN…
--- 0.63
RT @i24NEWS_EN: US tries to defuse South Chin

In [20]:
word_embedding

array([[ 0.00811615,  0.33195242,  0.02794582, ...,  0.02337269,
         0.37292427,  0.02534793],
       [ 0.43378973,  0.16321877, -0.44235185, ..., -0.32641962,
         0.11443402,  0.337322  ],
       [ 0.30983815,  0.45335525, -0.35523674, ..., -0.66587758,
         0.72239882, -0.09830879],
       ..., 
       [ 0.27629414,  0.24649541, -0.75071871, ..., -0.72112411,
         0.20808861,  0.2646122 ],
       [ 0.13015598, -0.01973688, -0.14896594, ...,  0.00254449,
         0.04784201,  0.33622169],
       [-0.09753007,  0.11830678, -0.08986864, ..., -0.11848716,
         0.09942216,  0.12464163]], dtype=float32)

In [21]:
word_embedding_norm = word_embedding / np.linalg.norm(word_embedding, axis=1, keepdims=True)

In [22]:
word_to_vocab = {w: i for i, w in enumerate(vocab)}

def e(s):
    return word_embedding[word_to_vocab[s]]

In [23]:
turnbull = e('Turnbull')
shorten = e('Shorten')
liberal = e('Liberal')
labor = e('Labor')

word = turnbull - liberal + labor
# word = shorten - labor + liberal

# word = e('plebiscite') - e('gay') - e('straight')

# word = e('Liberal')
# word = e('Labor')
# word = e('Greens')
# word = e('plebiscite')
# word = e('gay')
# word = e('sexism')
# word = e('budget')
# word = e('tax')
# word = e('debt')
# word = e('unemployment')
# word = e('foreign')
# word = e('TPP')
# word = e('anger')
# word = e('pollution')

word_norm = word / np.linalg.norm(word)
dist = np.sum(word_norm * word_embedding_norm, axis=1)
dist_sort_idx = np.argsort(dist)[::-1]
for i in dist_sort_idx[:25]:
    t = vocab[i]
    d = dist[i]
    print(np.round(d, 2), t)

0.73 Turnbull
0.68 Labor
0.49 @theheraldsun
0.47 Joyce
0.47 Coalition
0.46 @AdamBandt
0.45 name
0.45 Europe
0.44 complete
0.44 1/3
0.44 lifestyle
0.43 quotes
0.43 Mannoun
0.43 Cafe
0.43 VIC
0.42 legislate
0.42 bill
0.42 Oppn
0.42 Australia
0.42 mum
0.42 @AustralianLabor
0.42 Trump
0.41 mths
0.41 issue
0.41 Ovarian
